In [1]:
!pip install torchlibrosa loguru

     |████████████████████████████████| 57 kB 2.8 MB/s 


# New Section

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch.nn as nn
import cv2
from torchvision import models
import torch.nn.functional as F
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation
import pandas as pd
import torch
import librosa, librosa.display
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation
import numpy as np
import os
import pickle
import csv



import argparse
from tqdm import tqdm
from loguru import logger
from pathlib import Path
from itertools import chain
from re import sub

In [3]:
!wget https://zenodo.org/record/4783391/files/clotho_captions_evaluation.csv

--2021-12-25 05:44:25--  https://zenodo.org/record/4783391/files/clotho_captions_evaluation.csv
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361995 (354K) [text/plain]
Saving to: ‘clotho_captions_evaluation.csv’

clotho_captions_eva 100%[===================>] 353.51K   824KB/s    in 0.4s    

2021-12-25 05:44:27 (824 KB/s) - ‘clotho_captions_evaluation.csv’ saved [361995/361995]



In [6]:
basepath = '/content/drive/MyDrive/Upwork/eval_100'
df = pd.read_csv('/content/clotho_captions_evaluation.csv')
file_name = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
for filename in os.listdir(basepath):
  file_name.append(filename)
  c1.append(df[df["file_name"] == filename]['caption_1'].iloc[0])
  c2.append(df[df["file_name"] == filename]['caption_2'].iloc[0])
  c3.append(df[df["file_name"] == filename]['caption_3'].iloc[0])
  c4.append(df[df["file_name"] == filename]['caption_4'].iloc[0])
  c5.append(df[df["file_name"] == filename]['caption_5'].iloc[0])

data = pd.DataFrame({'file_name':file_name, 'caption_1':c1, 'caption_2':c2, 'caption_3':c3, 'caption_4':c4, 'caption_5':c5})
data.to_csv('/content/new_audio_data.csv')

In [7]:
def write_csv_file(csv_obj, file_name):
    with open(file_name, 'w') as f:
        writer = csv.DictWriter(f, csv_obj[0].keys())
        writer.writeheader()
        writer.writerows(csv_obj)
    print(f'Write to {file_name} successfully.')

def load_csv_file(file_name):
    with open(file_name, 'r') as f:
        csv_reader = csv.DictReader(f)
        csv_obj = [csv_line for csv_line in csv_reader]
    return csv_obj

def load_picke_file(file_name):
    with open(file_name, 'rb') as f:
        pickle_obj = pickle.load(f)
    return pickle_obj

def write_pickle_file(obj, file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(obj, f)
    print(f'Write to {file_name} successfully.')

In [8]:
def create_dataset():
    inner_logger = logger
    inner_logger.info('Loading csv files and process each caption.')

    dev_csv = load_csv_file('/content/new_audio_data.csv')
    # val_csv = load_csv_file('/content/clotho_captions_validation.csv')
    # eval_csv = load_csv_file('/content/clotho_captions_evaluation.csv')
    caption_fields = ['caption_{}'.format(i) for i in range(1, 6)]
    for csv_item in chain(dev_csv):
        ''' Process each caption'''
        captions = [_sentence_process(csv_item[caption_field], add_specials=True) for caption_field in caption_fields]
        [csv_item.update({caption_field: caption})
         for caption_field, caption in zip(caption_fields, captions)]
    inner_logger.info('Done!')
    words_list = load_picke_file('/content/drive/MyDrive/Upwork/new_words_list.p')

    for split_data in [(dev_csv, 'development')]:
        split_csv = split_data[0]
        split_name = split_data[1]
        split_dir = Path('data/data_splits', split_name)
        split_dir.mkdir(parents=True, exist_ok=True)

        audio_dir = Path('/content/drive/MyDrive/Upwork/eval_100')
        inner_logger.info(f'Creating the {split_name} split.')
        _create_split_data(split_csv, split_dir, audio_dir, words_list)
        inner_logger.info('Done')
        audio_number = len(os.listdir(str(audio_dir)))
        data_number = len(os.listdir(str(split_dir)))

        inner_logger.info('{} audio files in {}.'.format(audio_number, split_name))
        inner_logger.info('{} data files in {}'.format(data_number, split_name))
        inner_logger.info('{} data files per audio.'.format(data_number / audio_number))

    inner_logger.info('Dataset created')


def _create_vocabulary(captions):
    words_list = []
    vocabulary = []
    for caption in captions:
        caption_words = caption.strip().split()
        vocabulary.extend(caption_words)
    words_list = list(set(vocabulary))
    words_list.sort(key=vocabulary.index)
    words_freq = [vocabulary.count(word) for word in words_list]

    return words_list, words_freq


def _sentence_process(sentence, add_specials=False):
    sentence = sentence.lower()
    if add_specials:
        sentence = '<sos> {} <eos>'.format(sentence)
    # remove any forgotten space before punctuation and double space
    sentence = sub(r'\s([,.!?;:"](?:\s|$))', r'\1', sentence).replace('  ', ' ')
    # remove punctuations
    sentence = sub('[,.!?;:\"]', ' ', sentence).replace('  ', ' ')
    return sentence


def _create_split_data(split_csv, split_dir, audio_dir, words_list):
    sr = 22050#args.sr
    n_fft = 1024#args.n_fft
    hop_length = 512#args.hop_length
    n_mels = 64#args.n_mels
    window = 'hann'#args.window

    caption_fields = ['caption_{}'.format(i) for i in range(1, 6)]
    file_name_template = 'clotho_file_{audio_file_name}_{caption_index}.npy'
    for csv_entry in tqdm(split_csv, total=len(split_csv)):
        audio_file_name = csv_entry['file_name']
        audio, _ = librosa.load(audio_dir.joinpath(audio_file_name), sr=sr)
        feature = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft,
                                                 hop_length=hop_length,
                                                 n_mels=n_mels, window=window)
        feature = librosa.power_to_db(feature).T
        for caption_index, caption_field in enumerate(caption_fields):
            caption = csv_entry[caption_field]
            caption_words = caption.strip().split()
            words_indexs = [words_list.index(word) for word in caption_words]
            np_rec_array = np.rec.array(np.array(
                (audio_file_name, audio, feature, caption, caption_index, np.array(words_indexs)),
                dtype=[
                    ('file_name', 'U{}'.format(len(audio_file_name))),
                    ('audio_data', np.dtype(object)),
                    ('feature', np.dtype(object)),
                    ('caption', 'U{}'.format(len(caption))),
                    ('caption_index', 'i4'),
                    ('words_indexs', np.dtype(object))
                ]
            ))
            # save the numpy object
            file_name = str(split_dir.joinpath(file_name_template.format(
                                audio_file_name=  audio_file_name, caption_index=caption_index)))
            np.save(file_name, np_rec_array)

if __name__ == '__main__':
    create_dataset()

2021-12-25 05:45:38.251 | INFO     | __main__:create_dataset:3 - Loading csv files and process each caption.
2021-12-25 05:45:38.260 | INFO     | __main__:create_dataset:14 - Done!
2021-12-25 05:45:38.533 | INFO     | __main__:create_dataset:24 - Creating the development split.
  9%|▉         | 9/100 [00:14<02:29,  1.64s/it]


KeyboardInterrupt: ignored